# **Data Cleaning**

## Objectives
---

* Train an ML pipeline.
* Tune the pipeline using Hyperparameter Optimisation.
* Define the best features for predicting SalePrice.

## Steps
---

* Load data from the cleaned data outputs
* Perform a Grid Search CV with SKlearn.

## Import Necessary Packages
---

In [ ]:
import numpy as np
import pandas as pd

## Change working directory
---

First we set the working directory for the notebook.

In [ ]:
import os
current_dir = os.getcwd()
current_dir
os.chdir(os.path.dirname(current_dir))
current_dir = os.getcwd()
print(current_dir)

## Load Data
---

Load the data in from the outputs section of the last notebook. We can drop the SalePrice column as it is the target varialbe in this study.

In [ ]:
import pandas as pd
df = pd.read_csv('inputs/datasets/raw/house-price-20211124T154130Z-001/house-price/house_prices_records.csv')
df_clean = pd.read_csv("outputs/datasets/cleaned/clean_house_price_records.csv")
train_set = pd.read_csv("outputs/datasets/cleaned/train_set.csv")
test_set = pd.read_csv("outputs/datasets/cleaned/test_set.csv")

Change objects to categories.

In [ ]:
categorical_variables = df.select_dtypes(include=['object']).columns
df[categorical_variables] = df[categorical_variables].astype('category')
df_clean[categorical_variables] = df_clean[categorical_variables].astype('category')
train_set[categorical_variables] = train_set[categorical_variables].astype('category')
test_set[categorical_variables] = test_set[categorical_variables].astype('category')

Load pipelines from outputs.

In [ ]:
import joblib
data_cleaning_pipeline = joblib.load("outputs/pipelines/data_cleaning_pipeline.pkl")
feature_engineering_pipeline = joblib.load("outputs/pipelines/feature_engineering_pipeline.pkl")

In [ ]:
data_cleaning_pipeline

In [ ]:
feature_engineering_pipeline

Define complete pipeline for Optimization. This function returns a base pipeline that we can modify with different models.

In [ ]:
from sklearn.pipeline import Pipeline

### Feat Selection
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler


### ML algorithms
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
# from xgboost import XGBRegressor


def PipelineOptimization(model):
    pipeline_base = Pipeline([
        ('data_cleaning_pipeline', data_cleaning_pipeline),
        ('feature_engineering_pipeline', feature_engineering_pipeline),
        ('FeatureScaling', StandardScaler()),
        ('FeatureSelection', SelectFromModel(model)),
        ('model', model)
        ])
    return pipeline_base


## Hyperparameter Optimization
---

This is a custom class taken from the codeinstitute modules regarding machine learning. It allows us to loop through different algorithms and hyperparameters to find the best performing combinations for a given pipeline.

In [ ]:
from sklearn.model_selection import GridSearchCV

class HyperparameterOptimizationSearch:

    def __init__(self, models, params):
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}

    def fit(self, X, y, cv, n_jobs, verbose=1, scoring=None, refit=False):
        for key in self.keys:
            print(f"\nRunning GridSearchCV for {key} \n")
            model = PipelineOptimization(self.models[key])

            params = self.params[key]
            gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs, verbose=verbose, scoring=scoring)
            gs.fit(X,y)
            self.grid_searches[key] = gs    

    def score_summary(self, sort_by='mean_score'):
        def row(key, scores, params):
            d = {
                 'estimator': key,
                 'min_score': min(scores),
                 'max_score': max(scores),
                 'mean_score': np.mean(scores),
                 'std_score': np.std(scores),
            }
            return pd.Series({**params,**d})

        rows = []
        for k in self.grid_searches:
            params = self.grid_searches[k].cv_results_['params']
            scores = []
            for i in range(self.grid_searches[k].cv):
                key = "split{}_test_score".format(i)
                r = self.grid_searches[k].cv_results_[key]        
                scores.append(r.reshape(len(params),1))

            all_scores = np.hstack(scores)
            for p, s in zip(params,all_scores):
                rows.append((row(k, s, p)))

        df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)
        columns = ['estimator', 'min_score', 'mean_score', 'max_score', 'std_score']
        columns = columns + [c for c in df.columns if c not in columns]
        return df[columns], self.grid_searches

We split the train and test data from the dataset.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
                                    df.drop(['SalePrice'], axis=1),
                                    df['SalePrice'],
                                    test_size=0.2,
                                    random_state=0
                                    )

print("* Train set:", X_train.shape, y_train.shape, "\n* Test set:",  X_test.shape, y_test.shape)


## Grid Search CV
---

* We first perform a search for the best Algorithm using default hyperparameters.
* We define the algorithms and the blank hyperparameters below.

In [ ]:
models_quick_search = {
    'LinearRegression': LinearRegression(),
    "DecisionTreeRegressor": DecisionTreeRegressor(random_state=0),
    "RandomForestRegressor": RandomForestRegressor(random_state=0),
    "ExtraTreesRegressor": ExtraTreesRegressor(random_state=0),
    "AdaBoostRegressor": AdaBoostRegressor(random_state=0),
    "GradientBoostingRegressor": GradientBoostingRegressor(random_state=0),
    "XGBRegressor": XGBRegressor(random_state=0),
}

params_quick_search = {
    'LinearRegression': {},
    "DecisionTreeRegressor": {},
    "RandomForestRegressor": {},
    "ExtraTreesRegressor": {},
    "AdaBoostRegressor": {},
    "GradientBoostingRegressor": {},
    "XGBRegressor": {},
}

We perform the initial search.

In [ ]:
search = HyperparameterOptimizationSearch(models=models_quick_search, params=params_quick_search)
search.fit(X_train, y_train, scoring='r2', n_jobs=-1, cv=5)

Evaluating the results shows the best Regressor as the ExtraTreesRegressor.

In [ ]:
grid_search_summary, grid_search_pipelines = search.score_summary(sort_by='mean_score')
grid_search_summary

## Results
---

* An R2 score of at least 0.75 is required as per the business requirements.
* The ExtraTreesRegressor meets this requirement at a mean score of 0.79
* We can now tune the Hyperparameters to further raise the score.

## Extensive search
---

We can now perform an extensive Hyperparameter search using the ExtraTreesRegressor algorithm.

### Define Model Parameters

In [ ]:
models_search = {
    "ExtraTreesClassifier": ExtraTreesRegressor(random_state=0),
    'LinearRegression': LinearRegression(),
    }

# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html
params_search = {
    "ExtraTreesClassifier":{"model__n_estimators": [20,50],
                            'model__max_depth': [6,18,None],
                        },
    'LinearRegression': {},
}


In [ ]:
search = HyperparameterOptimizationSearch(models=models_search, params=params_search)
search.fit(X_train, y_train, scoring='r2', n_jobs=-1, cv=8)

We can see marginal improvements from this optimization.

In [ ]:
grid_search_summary, grid_search_pipelines = search.score_summary(sort_by='mean_score')
grid_search_summary

## Fit Best Result to Model
---

We can now fit the ExtraTreesClassifier to the Model with its tuned hyperparameters.

In [ ]:
best_model = grid_search_summary.iloc[0,0]
best_model

Define best regressor parameters.

In [ ]:
best_parameters = grid_search_pipelines[best_model].best_params_
best_parameters

In [ ]:
best_regressor_pipeline = grid_search_pipelines[best_model].best_estimator_
best_regressor_pipeline

## Assess Feature Importance
---

We can try to make the model more efficient by assessing the feature importance and dropping features that don't impact our final R2 score significantly.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

%matplotlib inline

data_cleaning_feat_eng_steps = 2 
columns_after_data_cleaning_feat_eng = (Pipeline(best_regressor_pipeline.steps[:data_cleaning_feat_eng_steps])
                                        .transform(X_train)
                                        .columns)

best_features = columns_after_data_cleaning_feat_eng[best_regressor_pipeline['FeatureSelection'].get_support()].to_list()

# create DataFrame to display feature importance
df_feature_importance = (pd.DataFrame(data={
          'Feature': columns_after_data_cleaning_feat_eng[best_regressor_pipeline['FeatureSelection'].get_support()],
          'Importance': best_regressor_pipeline['model'].feature_importances_})
  .sort_values(by='Importance', ascending=False)
  )

# Most important features statement and plot
print(f"* These are the {len(best_features)} most important features in descending order. "
      f"The model was trained on them: \n{best_features}")

df_feature_importance.plot(kind='bar',x='Feature',y='Importance')
plt.show()

## Evaluate Pipeline Performance
---

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error 
import numpy as np

def regression_performance(X_train, y_train, X_test, y_test,pipeline):
	print("Model Evaluation \n")
	print("* Train Set")
	regression_evaluation(X_train,y_train,pipeline)
	print("* Test Set")
	regression_evaluation(X_test,y_test,pipeline)

def regression_evaluation(X,y,pipeline):
  prediction = pipeline.predict(X)
  print('R2 Score:', r2_score(y, prediction).round(3))  
  print('Mean Absolute Error:', mean_absolute_error(y, prediction).round(3))  
  print('Mean Squared Error:', mean_squared_error(y, prediction).round(3))  
  print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y, prediction)).round(3))
  print("\n")


def regression_evaluation_plots(X_train, y_train, X_test, y_test, pipeline, alpha_scatter=0.5):
  pred_train = pipeline.predict(X_train)
  pred_test = pipeline.predict(X_test)


  fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,6))
  sns.scatterplot(x=y_train , y=pred_train, alpha=alpha_scatter, ax=axes[0])
  sns.lineplot(x=y_train , y=y_train, color='red', ax=axes[0])
  axes[0].set_xlabel("Actual")
  axes[0].set_ylabel("Predictions")
  axes[0].set_title("Train Set")

  sns.scatterplot(x=y_test , y=pred_test, alpha=alpha_scatter, ax=axes[1])
  sns.lineplot(x=y_test , y=y_test, color='red', ax=axes[1])
  axes[1].set_xlabel("Actual")
  axes[1].set_ylabel("Predictions")
  axes[1].set_title("Test Set")

  plt.show()

In [ ]:
regression_performance(X_train, y_train, X_test, y_test, best_regressor_pipeline)
regression_evaluation_plots(X_train, y_train, X_test, y_test, best_regressor_pipeline)

## Applying a Regressor with PCA
---

* We may be able to further improve the model by using PCA.

In [ ]:
pipeline = PipelineOptimization(model=ExtraTreesRegressor())
pipeline_pca = Pipeline(pipeline.steps[:3])
df_pca = pipeline_pca.fit_transform(df.drop(['SalePrice'],axis=1), df['SalePrice'])

print(df_pca.shape,'\n', type(df_pca))

We can see that a 7 component analysis contains 76.4% of the data. +-2 on this Value cahncged the output R2 score marginally.

In [ ]:
import numpy as np
from sklearn.decomposition import PCA

n_components = 7

def pca_components_analysis(df_pca,n_components): 
    pca = PCA(n_components=n_components).fit(df_pca)
    x_PCA = pca.transform(df_pca) # array with transformed PCA

    ComponentsList = ["Component " + str(number) for number in range(n_components)]
    dfExplVarRatio = pd.DataFrame(
        data= np.round(100 * pca.explained_variance_ratio_ ,3),
        index=ComponentsList,
        columns=['Explained Variance Ratio (%)'])

    dfExplVarRatio['Accumulated Variance'] = dfExplVarRatio['Explained Variance Ratio (%)'].cumsum() 

    PercentageOfDataExplained = dfExplVarRatio['Explained Variance Ratio (%)'].sum()

    print(f"* The {n_components} components explain {round(PercentageOfDataExplained,2)}% of the data \n")
    plt.figure(figsize=(12,5))
    sns.lineplot(data=dfExplVarRatio,  marker="o")
    plt.xticks(rotation=90)
    plt.yticks(np.arange(0, 110, 10))
    plt.show()

pca_components_analysis(df_pca=df_pca,n_components=n_components)

## Rewrite ML Pipeline with PCA
---

We re-write the pipeline, this time without feature selection, instead we use our PCA with 7 components.

In [ ]:
from sklearn.decomposition import PCA

selection_method = 'cardinality'
corr_method = 'spearman'
n_components = 7

def PipelineOptimization(model):
    pipeline_base = Pipeline([
        ('data_cleaning_pipeline', data_cleaning_pipeline),
        ('feature_engineering_pipeline', feature_engineering_pipeline),
        ('FeatureScaling', StandardScaler()),
        ("PCA", PCA(n_components=n_components, random_state=0)),
        ('model', model)
        ])
    return pipeline_base


In [ ]:
print("* Train set:", X_train.shape, y_train.shape, "\n* Test set:",  X_test.shape, y_test.shape)


Using Standard Hyperparameters

In [ ]:
models_quick_search = {
    'LinearRegression': LinearRegression(),
    "DecisionTreeRegressor": DecisionTreeRegressor(random_state=0),
    "RandomForestRegressor": RandomForestRegressor(random_state=0),
    "ExtraTreesRegressor": ExtraTreesRegressor(random_state=0),
    "AdaBoostRegressor": AdaBoostRegressor(random_state=0),
    "GradientBoostingRegressor": GradientBoostingRegressor(random_state=0),
    "XGBRegressor": XGBRegressor(random_state=0),
}

params_quick_search = {
    'LinearRegression': {},
    "DecisionTreeRegressor": {},
    "RandomForestRegressor": {},
    "ExtraTreesRegressor": {},
    "AdaBoostRegressor": {},
    "GradientBoostingRegressor": {},
    "XGBRegressor": {},
}

Perform quick search

In [ ]:
quick_search = HyperparameterOptimizationSearch(models=models_quick_search, params=params_quick_search)
quick_search.fit(X_train, y_train, scoring='r2', n_jobs=-1, cv=5)

Check results

In [ ]:
grid_search_summary, grid_search_pipelines = quick_search.score_summary(sort_by='mean_score')
grid_search_summary

In [ ]:
models_search = {
    "RandomForestRegressor":RandomForestRegressor(random_state=0),
    "ExtraTreesClassifier": ExtraTreesRegressor(random_state=0),
}

params_search = {
    "ExtraTreesClassifier":{"model__n_estimators": [20,50],
                            'model__max_depth': [6,18,None],
                        },
    "RandomForestRegressor":{
        'model__n_estimators': [100,700],
        'model__max_depth': [6,18,None],
    }
}

In [ ]:
search = HyperparameterOptimizationSearch(models=models_search, params=params_search)
search.fit(X_train, y_train, scoring = 'r2', n_jobs=-1, cv=5)

Check the results.

In [ ]:
grid_search_summary, grid_search_pipelines = search.score_summary(sort_by='mean_score')
grid_search_summary

Extract the best model

In [ ]:
best_model = grid_search_summary.iloc[0,0]
best_model

Check best parameters.

In [ ]:
grid_search_pipelines[best_model].best_params_


Define the best regressor model.

In [ ]:
best_regressor_pipeline_pca = grid_search_pipelines[best_model].best_estimator_
best_regressor_pipeline_pca

## Evaluation
---

Evaluate the PCA regressor on the train and test set.

In [ ]:
regression_performance(X_train, y_train, X_test, y_test,best_regressor_pipeline_pca)
regression_evaluation_plots(X_train, y_train, X_test, y_test,
                            best_regressor_pipeline_pca)

## Subset Best Features
---

We can now subset the best features revealed by the PCA.

In [ ]:
X_train = X_train.filter(best_features)
X_test = X_test.filter(best_features)

print("* Train set:", X_train.shape, y_train.shape, "\n* Test set:", X_test.shape, y_test.shape)
X_train.head()

We then rewrite the pipeline again.

In [ ]:
from feature_engine.transformation import PowerTransformer
from feature_engine.transformation import YeoJohnsonTransformer
from feature_engine.transformation import LogTransformer
def PipelineOptimization(model):
    pipeline_base = Pipeline([
        ('LogTransform', LogTransformer(variables=['GrLivArea',])),
        ('YeoJohnsonTransform', YeoJohnsonTransformer(variables=['YearBuilt',])),
        ('PowerTransform', PowerTransformer(variables=['GarageArea',])),
        ('FeatureScaling', StandardScaler()),
        ('model', model)
        ])
    return pipeline_base

We evaluate the pipeline using the previously established best algorithms.

In [ ]:
models_search = {
    "RandomForestRegressor":RandomForestRegressor(random_state=0),
    "ExtraTreesClassifier": ExtraTreesRegressor(random_state=0),
}

We run the extended search using the recommended hyperparameters.

In [ ]:
params_search = {
    "ExtraTreesClassifier":{"model__n_estimators": [20,50],
                            'model__max_depth': [6,18,None],
                        },
    "RandomForestRegressor":{
        'model__n_estimators': [100,700],
        'model__max_depth': [6,18,None],
    }
}

In [ ]:
search = HyperparameterOptimizationSearch(models=models_search, params=params_search)
search.fit(X_train, y_train, scoring='r2', n_jobs=-1, cv=5)

In [ ]:
grid_search_summary, grid_search_pipelines = search.score_summary(sort_by='mean_score')
grid_search_summary

We can now extract the best Algorithm and set of Hyperparameters.

In [ ]:
best_model = grid_search_summary.iloc[0,0]
best_model

In [ ]:
best_regressor_pipeline = grid_search_pipelines[best_model].best_estimator_
best_regressor_pipeline

In [ ]:
best_parameters = grid_search_pipelines[best_model].best_params_
best_parameters

## Final Feature Importance Assessment
---

We now assess the feature importance, using the refitted pipieline and features.

In [ ]:
data_cleaning_feat_eng_steps = 3 # how many data cleaning and feature engineering does your pipeline have?
columns_after_data_cleaning_feat_eng = (Pipeline(best_regressor_pipeline.steps[:data_cleaning_feat_eng_steps])
                                        .transform(X_train)
                                        .columns)

best_features = columns_after_data_cleaning_feat_eng

# create DataFrame to display feature importance
df_feature_importance = (pd.DataFrame(data={
          'Feature': columns_after_data_cleaning_feat_eng,
          'Importance': best_regressor_pipeline['model'].feature_importances_})
  .sort_values(by='Importance', ascending=False)
  )

# Most important features statement and plot
print(f"* These are the {len(best_features)} most important features in descending order. "
      f"The model was trained on them: \n{best_features}")

df_feature_importance.plot(kind='bar',x='Feature',y='Importance')
plt.show()

We can see from the above graphic that by far, OverallQual has the most prominent effect on the prediction of the house prices.

In [ ]:
regression_performance(X_train, y_train, X_test, y_test, best_regressor_pipeline)
regression_evaluation_plots(X_train, y_train, X_test, y_test, best_regressor_pipeline)

* We have achieved a final score of 0.894 on the train set and 0.811 on the test set.
* This indicates that the model is slightly overfitting.
* We have achieved our business goal and delivered a tuned and optimized model that provides an R2 score of 0.811 on the test data.

## Save Model and Dataset
---

* We can now save the model and the dataset to the outputs folder.
* We will also save the Features Importance plot.

In [ ]:
import os

version = 'v1'
file_path = f'outputs/pipelines/predict_saleprice/{version}'

try:
  os.makedirs(name=file_path)
except Exception as e:
  print(e)

### Train Set


**Features**

In [ ]:
print(X_train.shape,)
X_train.head(5)

In [ ]:
X_train.to_csv(f"{file_path}/X_train.csv", index=False)


**Target**

In [ ]:
print(y_train.shape)
y_train.head()

In [ ]:
y_train.to_csv(f"{file_path}/y_train.csv", index=False)


### Test Set

**Features**

In [ ]:
print(X_test.shape)
X_test.head()

In [ ]:
X_test.to_csv(f"{file_path}/X_test.csv", index=False)


**Target**

In [ ]:
print(y_test.shape)
y_test.head()

In [ ]:
y_test.to_csv(f"{file_path}/y_test.csv", index=False)


## Model Pipeline

In [ ]:
best_regressor_pipeline

In [ ]:
import joblib
joblib.dump(value=best_regressor_pipeline, filename=f"{file_path}/best_regressor_pipeline.pkl")

### Feature Importance Plot

In [ ]:
df_feature_importance.to_csv(f"{file_path}/feature_importance.csv", index=False)

In [ ]:
df_feature_importance.plot(kind='bar',x='Feature',y='Importance')
plt.savefig(f'{file_path}/feature_importance.png', bbox_inches='tight')

In [ ]:

pred_train = best_regressor_pipeline.predict(X_train)
pred_test = best_regressor_pipeline.predict(X_test)
alpha_scatter=0.5

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,6))
sns.scatterplot(x=y_train , y=pred_train, alpha=alpha_scatter, ax=axes[0])
sns.lineplot(x=y_train , y=y_train, color='red', ax=axes[0])
axes[0].set_xlabel("Actual")
axes[0].set_ylabel("Predictions")
axes[0].set_title("Train Set")

sns.scatterplot(x=y_test , y=pred_test, alpha=alpha_scatter, ax=axes[1])
sns.lineplot(x=y_test , y=y_test, color='red', ax=axes[1])
axes[1].set_xlabel("Actual")
axes[1].set_ylabel("Predictions")
axes[1].set_title("Test Set")

plt.savefig(f'{file_path}/regression_evaluation_plots.png', bbox_inches='tight')

## Conclusion
---

* The Regressor for predicting the House Price has been selected as the RandomForrestRegressor.
* The Feature Selection Pipeline performed slightly better, achieving a result of 0.836 on the test set and 0.861 on the train set.
* Using either Pipeline, the business requirement has veen met, and an R2 score of greater than 0.75 has been achieved.